In [1]:
import pandas as pd 
import numpy as np 
import requests
import random
import shutil 
import os
#from fastai.vision.all import *

In [2]:


def get_random_coordinates(n=10):
    #Get list of random coordinates from Germany
    lat_min_max = [47.2701114, 55.099161]
    lon_min_max = [5.8663153, 15.0419319]
    lat_lon_noWT = pd.DataFrame([(random.uniform(*lat_min_max), random.uniform(*lon_min_max)) for _ in range(n)], columns =["lat", "lon"])
    return lat_lon_noWT

def get_WT_coordinates():
    return pd.read_csv("data/wind_new.csv")

def get_POWER_coordinates():
    return pd.read_csv("data/strom_new.csv")


In [3]:
def load_images_mapbox(df, prefix, path="default", zoom=14.5, rotation=0, width=256, random_n=None):

    if random_n:
        df = df.sample(random_n)

    height = width

    lat_lon = list(df[["lat", "lon"]].itertuples(name=None))
    token = ''
    
    # {zoom}_{rotation}_{width}
    path = f"./data/images/{path}"

    os.makedirs(path, exist_ok=True)

    for index, lat, lon in lat_lon:
        r = requests.get('https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/'\
                        f'{lon},{lat},{zoom},{rotation}/{width}x{height}@2x'\
                        f'?access_token={token}', stream =True)
        print(f"{index}: (lat, lon):{lat}, {lon}")
            
        with open(f'{path}/{prefix}_{index}.png', 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)

def load_images_google(df, prefix, width=256, path="default", random_n=None):

    if random_n:
        df = df.sample(random_n)

    height = width

    # {zoom}_{rotation}_{width}
    path = f"./data/images/{path}"

    os.makedirs(path, exist_ok=True)

    key = ""
    lat_lon = list(df[["lat", "lon"]].itertuples(name=None))

    for index, lat, lon in lat_lon:
        print(f"{index}: (lat, lon):{lat}, {lon}")
        r = requests.get("https://maps.googleapis.com/maps/api/staticmap?"\
                    f"center={lat},{lon}&zoom=18&size={width}x{height}&maptype=satellite&key={key}", stream =True)
        with open(f'{path}/{prefix}_{index}_google.png', 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)

In [4]:
def load_training_images():
    wind = get_WT_coordinates()
    power = get_POWER_coordinates()
    random = get_random_coordinates(n=155)

    load_images_mapbox(wind, prefix="WT", path="train")
    load_images_mapbox(power, prefix="NWT", path="train")
    load_images_mapbox(random, prefix="NWT", path="train")